UI Code for 789A-4 Scan Controller of McPherson Monochromator

This code defines functions that are used to issue serial commands to the scan controller from a computer. It also creates the user interface for accessing those commands and entering different values for use of experiments.

I. Libraries
II. Connect
III. CheckStatus and Homing
IV. GoTo Scan
V. Basic Scan
VI. Advanced Scan
VII. UI Configuration

I. Libraries

In [1]:
import codecs
import time
import serial
import numpy as np
from tkinter import *
import tkinter as tk
from icecream import ic

II. Connect
Serial settings for Scan Controller communication. Opens and closes communications to allow code to be more simplistic instead of repeating ser.open(), ser.close() commands.

In [2]:
#function to establish serial values for communication with monochromator and computer
def connect(event):
    try:
        ser = serial.Serial('COM3', 
                            baudrate = 9600, 
                            timeout = None,
                            xonxoff = True,
                            parity = serial.PARITY_NONE,
                            stopbits = serial.STOPBITS_ONE,
                            bytesize = serial.EIGHTBITS,
                            )
        ser.close()
        print("Connection established")
    except:
        print("Error, connection could not be established")
    finally:
        print("Close the connection before exiting the program")
    return

III. Check Status and Homing
Outputs the status of the limit switch, which tells the user if the Scan Controller is greater or less than the home wavelength of 631.26nm. A status of 0 or 2 means that the Controller is greater than home. A status of 32 or 34 is less than home. Home function enables the home circuit to configure wavelength; then checks the limit status. From that status value it either moves at a constant velocity and continues to check the limit status. Once the limit status changes from passing home wavelength, scanning stops and fine movement to reach home commences and mechanical backlash is accounted for. Finally, it disables the home circuit and then exits the program. 

In [3]:
#function to check if the monochromator is greater or less than home wavelength(631.26nm)
def check_status(wait_time=1):
    try:
        ser = serial.Serial('COM3', 
                                baudrate = 9600, 
                                timeout = None,
                                xonxoff = True,
                                parity = serial.PARITY_NONE,
                                stopbits = serial.STOPBITS_ONE,
                                bytesize = serial.EIGHTBITS,
                                )
        ser.close()
        time.sleep(wait_time) #gives command time to readout full message from serial reciever
        ser.open()
        ser.write(b'] \r'); #check limit status
        
        s = ser.read_until(size=None) #reads output from serial until there is no more data transmitted
        ser.close()
        stat_now = codecs.decode(s) #decodes info from serial to m-23000 to confirm movement
        time.sleep(wait_time)
        stat_now = int(str(stat_now[4:])) #should slice output to only be the interger, not ]    0
    except:
        print("Limit Status Could Not Be Read") 
    finally:
        return stat_now #updates value for stat_now when calling stat_now = check_status()

#Function to home monochromator to 631.26nm
def home(event):
    try:
        #initialize connection of program and monochromator
        ser = serial.Serial('COM3', 
                                baudrate = 9600, 
                                timeout = None,
                                xonxoff = True,
                                parity = serial.PARITY_NONE,
                                stopbits = serial.STOPBITS_ONE,
                                bytesize = serial.EIGHTBITS,
                                )
        ser.close()
        ser.open()                            
        ser.write(b' \r'); 
        ser.read_until(size=None) 
        ser.close()
        print("communication initialized")

        #enable home circuit to configure to home wavelength
        ser.open()
        ser.write(b'A8 \r'); 
        ser.read_until(size=None)
        ser.close()
        print("home circuit enabled")

        #check the limit status before starting movement, so the code know to scan up or down
        stat_now = check_status()
        print("initial limit status is:",stat_now)
        
        if stat_now < 32: #above home stat_now=0, above home and moving stat_now=2
            ser.open()
            ser.write(b'm-23000 \r'); #move at constant vel. of 23KHz decreasing wavelength
            ser.read_until(size=None)
            ser.close() 
            print("decreasing wavelength at a rate of 23KHz")

            while stat_now < 32: #once scan passes home stat_now should switch to 34
                stat_now = check_status() 
                print("limit status is:",stat_now)

                if stat_now > 32: #below home stat_now=32, below home and moving stat_now=34
                    ser.open()
                    ser.write(b'@ \r'); #soft stop
                    ser.read_until(size=None)
                    ser.close()
                    print("scan controller stopped")

                    time.sleep(0.8)
                    ser.open()
                    ser.write(b'-108000 \r'); #turns motor for 3 rev., subtracts 12nm
                    ser.read_until(size=None)
                    ser.close()
                    print("decreasing wavelength for 3 revolutions")

                    time.sleep(2)
                    ser.open()
                    ser.write(b'+72000 \r'); #turns motor for 2 rev., removes backlash adds 8nm
                    ser.read_until(size=None)
                    ser.close()
                    print("increasing wavelength for 2 revolutions")
                    
                    time.sleep(1)
                    ser.open()
                    ser.write(b'A24 \r'); #enable high accuracy circuit for fine movement
                    ser.read_until(size=None)
                    ser.close()
                    print("high accuracy circuit enabled")

                    ser.open()
                    ser.write(b'F4500,0 \r'); #find edge of home flag at 1000 steps/sec
                    ser.read_until(size=None)
                    ser.close()
                    print("finding edge of home flag at 4500KHz")

                    time.sleep(35) #time it takes to complete F1000,0 movement
                    ser.open()
                    ser.write(b'@ \r'); #soft stop
                    ser.read_until(size=None)
                    ser.close()
                    print("scan controller stopped")

                    ser.open()
                    ser.write(b'A0 \r'); #disables home circuit
                    ser.read_until(size=None)
                    ser.close()
                    print("disabled home circuit")

                    ser.open()
                    ser.write(b'P \r'); #exit program
                    ser.read_until(size=None)
                    ser.close()
                    print("exited program ")
                    print("finished homing, now at 631.16nm")

        if stat_now > 0: #below home wavelength stat_now=32, below home and moving stat_now=34
            ser.open()
            ser.write(b'm+23000 \r')
            ser.read_until(size=None)
            ser.close()
            print("increasing wavelength at a rate of 23KHz ")

            while stat_now > 2: #once scan passes home stat_now should switch to 2
                stat_now = check_status()
                print("limit status is:",stat_now)
                
                if stat_now < 32: #above home wavelength stat_now=0, above home and moving stat_now=2
                    ser.open()
                    ser.write(b'@ \r'); #soft stop
                    ser.read_until(size=None)
                    print("scan controller stopped")
                    ser.close()

                    time.sleep(0.8)    
                    ser.open()
                    ser.write(b'-108000 \r'); #decreasing wavelength for 3 motor revolutions
                    ser.read_until(size=None)
                    ser.close()
                    print("decrease wavelength for 3 revolutions")

                    time.sleep(2)
                    ser.open()
                    ser.write(b'+72000 \r'); #removes backlash
                    ser.read_until(size=None)
                    ser.close()
                    print("increase wavelength for 2 revolutions")
                    
                    time.sleep(1)
                    ser.open()
                    ser.write(b'A24 \r'); #enable high accuracy circuit
                    ser.read_until(size=None)
                    ser.close()
                    print("high accuracy circuit enabled")

                    ser.open()
                    ser.write(b'F4500,0 \r'); #find edge of home flag at 1000 steps/sec
                    ser.read_until(size=None)
                    ser.close()
                    print("finding edge of home flag at 4500KHz ")

                    time.sleep(35)
                    ser.open()
                    ser.write(b'@ \r'); #soft stop
                    ser.read_until(size=None)
                    ser.close()
                    print("scan controller stopped")

                    ser.open()
                    ser.write(b'A0 \r'); #disable home circuit
                    ser.read_until(size=None)
                    ser.close()
                    print("disabled home circuit")

                    ser.open()
                    ser.write(b'P \r'); #exit program
                    ser.read_until(size=None)
                    ser.close()
                    print("exited program ")
                    print("finished homing, now at 631.16nm")
    except:
        print("Error reading limit status, could not determine distance from home.")
        #raise Exception if scan controller already at home
    finally:
        ser.open()
        ser.write(b'@ \r'); #soft stop
        ser.read_until(size=None)
        ser.close()
        print("scan controller stopped")

        ser.open()
        ser.write(b'P \r'); #exit program
        ser.read_until(size=None)
        ser.close()
        print("exited program")

IV. Go To Scan
Initializes Serial Communication between Scan Controller and computer. Enables home circuit. User input within limits, then takes the difference between one given wavelength and home wavelength. Converts difference to mechanical steps. Then to string and to bytes for serial command. Error handling for entering numbers with commas or no connection.

In [4]:
#scan to one certain wavelength, within range and input by user, converted to revolutions
def GoTo(event):
    try:
        ser = serial.Serial('COM3', 
                                baudrate = 9600, 
                                timeout = None,
                                xonxoff = True,
                                parity = serial.PARITY_NONE,
                                stopbits = serial.STOPBITS_ONE,
                                bytesize = serial.EIGHTBITS,
                                )
        ser.close()

        ser.open()                            
        ser.write(b' \r'); 
        ser.read_until(size=None) 
        ser.close()
        print("communication initialized")

        #enable home circuit to configure to home wavelength
        ser.open()
        ser.write(b'A8 \r'); 
        ser.read_until(size=None)
        ser.close()
        print("home circuit enabled")
        
        #user input
        try:
            wavelength = float(input("Enter the wavelength you want to scan to: "))
            uplim = 999.9 #nm
            lowlim = 0.1 #nm
            home = 631.26 #nm
            rev = 9000 #steps
            if wavelength > uplim: #monochromator upper limit 
                print("please enter a number lower than 999.9")
            if wavelength < lowlim: #monochromator lower limit
                print("please enter a number higher than 0.1")
            #equation to find difference between home position and new wavelength desired
            difference = wavelength - home 
            #eq for number of motor steps from home to desired wavelength, 9000 steps = 1 nm
            steps = difference * rev
            #take off fraction of a step for mechanical movement
            serialsteps = round(steps,0)
            #convert to string
            str = f'{serialsteps}' + ' \r'
            #convert string to byte for serial command for monochromator to read
            str_2_bytes = bytes(str, 'utf-8')
            if lowlim < wavelength < uplim:
                ser = serial.Serial('COM3', 
                                    baudrate = 9600, 
                                    timeout = None,
                                    xonxoff = True,
                                    parity = serial.PARITY_NONE,
                                    stopbits = serial.STOPBITS_ONE,
                                    bytesize = serial.EIGHTBITS,
                                    )
                ser.close()
                ser.open()                            
                ser.write(b' \r'); 
                ser.read_until(size=None) 
                ser.close()
                print("communication initialized")
                
                ser.open()
                ser.write(str_2_bytes); 
                ser.read_until(size=None)
                ser.close()
                print('scan controller is moving for', serialsteps,'revolutions')
                
                #wait command

                ser.open()
                ser.write(b'@ \r'); #soft stop
                ser.read_until(size=None)
                ser.close()
                print("scan controller stopped")

                #wait command, exposure time?
                
                ser.open()
                ser.write(b'P \r'); #exit program
                ser.read_until(size=None)
                ser.close()
                print("exited program")
        except: 
            print("Could not complete move command")
    except:
        print("Error establishing connection.")

V. Basic Scan
Function for completing scan with a beginning and end wavelength. Two inputs for user to enter wavelength values. Converts wavelengths to mechanical steps, then converts the steps to a string and then to bytes for serial communication to Scan Controller. 

In [5]:
def scanbasic(event):
    try:
        ser = serial.Serial('COM3', 
                                baudrate = 9600, 
                                timeout = None,
                                xonxoff = True,
                                parity = serial.PARITY_NONE,
                                stopbits = serial.STOPBITS_ONE,
                                bytesize = serial.EIGHTBITS,
                                )
        ser.close()
        
        ser.open()                            
        ser.write(b' \r'); 
        ser.read_until(size=None) 
        ser.close()
        print("communication initialized")

        #this function takes wl range and interval from user and scans the monochromator for a given hold time for each wl 
        #user input for start and end wavelength of monochromator scan from any given wavelength within limits
        try:
            wavelength_start = float(input("Enter the wavelength you want to scan to: "))
            wavelength_end = float(input("Enter the wavelength you want to scan to: "))
            #upper wavelength limit of mechanical step motor on monochromator
            uplim = 999.9 #nm
            #lower wavelength limit of mechanical step motor on monochromator
            lowlim = 0.1 #nm
            #home wavelength for monochromator
            home = 631.26 #nm
            #wavelength to mechanical step motor conversion 9000 step = 1 nm
            rev = 9000.0 #steps
            if wavelength_start > uplim: #monochromator upper limit error
                print("please enter a number lower than 999.9")
            if wavelength_start < lowlim: #monochromator lower limit error
                print("please enter a number higher than 0.1")
            if wavelength_end >  uplim: #monochromator upper limit error
                print("please enter a number lower than 999.9")
            if wavelength_end < lowlim: #monochromator lower limit error
                print("please enter a number higher than 0.1")
            #equation to find difference between home position and new wavelength desired
            difference_start = wavelength_start - home
            difference_end = wavelength_end - wavelength_start
            #conversion eq for number of steps from home to new desired wavelength, 9000 steps = 1 nm
            steps_start = difference_start * rev
            steps_end = difference_end * rev
            #take off fraction of a step for mechanical movement
            serialstart = round(steps_start,0)
            serialend = round(steps_end,0)
            #first wavelength convert to string and then convert string to byte for serial command
            if serialstart > 0:
                start = str('+' f'{serialstart}' + ' \r')
                str_2_bytestart = bytes(start, 'utf-8')
            if serialstart <= 0:
                start = str(f'{serialstart}' + ' \r')
                str_2_bytestart = bytes(start, 'utf-8')
            #second wavelength convert to string and then convert to byte for serial command
            if serialend > 0:
                end = str('+' f'{serialend}' + ' \r')
                str_2_byteend = bytes(end, 'utf-8')
            if serialend <= 0:
                end = str(f'{serialend}' + ' \r')
                str_2_byteend = bytes(end, 'utf-8')
            #if statement when input wavelength within limits that completes monochromator initial movement
            if lowlim < wavelength_start <  uplim:
                ser = serial.Serial('COM3', 
                                    baudrate = 9600, 
                                    timeout = None,
                                    xonxoff = True,
                                    parity = serial.PARITY_NONE,
                                    stopbits = serial.STOPBITS_ONE,
                                    bytesize = serial.EIGHTBITS,
                                    )
                ser.close()
                ser.open()                            
                ser.write(b' \r'); #initialize communication between monochromator and serial
                ser.read_until(size=None) 
                ser.close()
                print("communication initialized")
                
                ser.open()
                ser.write(str_2_bytestart); #value of steps to scan 
                ser.read_until(size=None)
                ser.close()
                print('scan controller is moving for', serialstart,'steps')

                #wait command, exposure time?

                ser.open()
                ser.write(b'@ \r'); #soft stop
                ser.read_until(size=None)
                ser.close()
                print("scan controller stopped")

                #wait command, exposure time?

            #if statement when input wavelength within limits that completes monochromator ending movement
            if lowlim < wavelength_end <  uplim:
                ser = serial.Serial('COM3', 
                                    baudrate = 9600, 
                                    timeout = None,
                                    xonxoff = True,
                                    parity = serial.PARITY_NONE,
                                    stopbits = serial.STOPBITS_ONE,
                                    bytesize = serial.EIGHTBITS,
                                    )
                ser.close()
                ser.open()                            
                ser.write(b' \r'); 
                ser.read_until(size=None) 
                ser.close()
                print("communication initialized")

                ser.open()
                ser.write(str_2_byteend); 
                ser.read_until(size=None)
                ser.close()
                print('scan controller is moving for', serialend, 'steps')

                #wait command

                ser.open()
                ser.write(b'@ \r'); #soft stop
                ser.read_until(size=None)
                ser.close()
                print("scan controller stopped")

                #wait command, exposure time?
                
                ser.open()
                ser.write(b'P \r'); #exit program
                ser.read_until(size=None)
                ser.close()
                print("exited program")
        except:
            print("Error completing beginning or end scan")
    except:
        print("Error establishing connection.")

VI.Advanced Scan
Takes up to ten user inputs for wavelengths to scan to in seccession. Takes up to ten user inputs for exposures to each previously entered wavelength. Creates an array of waves and exposure. For the first wave, takes the difference between the wave and home and converts it into mechanical steps and then converts into string and bytes pauses after movement for exposure. For the rest of the array, takes the difference between the second wavelength and first wavelength, continues to convert to mechanical steps and from string into bytes and pauses after for exposure. Repeats for difference between third and second wavelength and so on through the rest of the list. 

In [6]:
def scanadv(event):
    try:
        """This function assumes that the monochromator is already on the home position"""
        #user inputs for 8 wavelengths and exposures
        wavelist = np.array([input("Enter wavelength you want to scan to: ") for wave in range(8)])
        exposure = np.array([input("Enter exposure time for each wavelength: ") for exp in range(8)])
        #calibration wavelength for instrument
        home = 631.26 #nm
        rev = 9000 #steps = 1 nm conversion factor for wavelength to mechanical step, used for serial commands
        #indexes wavelist entry so first input can be used to find the difference between home wavelength and the first entry
        try:
            for idx,wave in enumerate(wavelist):  
                if idx == 0:
                    currentwave = home
                    diff_wave = float(wavelist[0])-currentwave
                    diff_step = diff_wave*rev
                    #for positive values of diff_step, add plus to value while converting to string and then bytes
                    if diff_step > 0:
                        diff_stepstr = str('+' f'{diff_step}' + ' \r')
                        diff_stepbyte = bytes(diff_stepstr, 'utf-8')
                    #for all other values of diff_step, keep value and convert to string and then bytes
                    if diff_step <= 0:
                        diff_stepstr = str(f'{diff_step}' + ' \r')
                        diff_stepbyte = bytes(diff_stepstr, 'utf-8')
                
                    #serial move command for diff_step to desired wavelength
                    ser = serial.Serial('COM3', 
                                    baudrate = 9600, 
                                    timeout = None,
                                    xonxoff = True,
                                    parity = serial.PARITY_NONE,
                                    stopbits = serial.STOPBITS_ONE,
                                    bytesize = serial.EIGHTBITS,
                                    )
                    ser.close()
                    ser.open()                            
                    ser.write(b' \r'); 
                    ser.read_until(size=None) 
                    ser.close()
                    print("serial communication initialized")

                    ser.open()
                    ser.write(diff_step); 
                    ser.read_until(size=None)
                    ser.close(diff_stepbyte)
                    print('scan controller is moving by', diff_step, 'steps')
                    print(f"Moving scan controller {diff_wave}nm to {wavelist[0]}nm")
                    print(f"Holding for current exposure {exposure[idx]} seconds")
                    print(f"The exposure for {wavelist[0]}nm was taken")

                    ser.open()
                    ser.write(b'@ \r'); #soft stop
                    ser.read_until(size=None)
                    ser.close()
                    print("scan controller stopped")
                    #update list and go to the next input in the list
                    currentwave = wavelist[idx+1]
                else:
                    #for other inputs in wavelist index find difference between the current wavelength and the previous input in the list
                    diff_wave = float(currentwave)-float(wavelist[idx-1])
                    diff_step = diff_wave*rev
                    #for positive values of diff_step, add plus sign and convert from string to bytes
                    if diff_step > 0:
                        diff_stepstr = str('+' f'{diff_step}' + ' \r')
                        diff_stepbyte = bytes(diff_stepstr, 'utf-8')
                    #for all other values of diff_step, keep the same, convert to string and then bytes
                    if diff_step <= 0:
                        diff_stepstr = str(f'{diff_step}' + ' \r')
                        diff_stepbyte = bytes(diff_stepstr, 'utf-8')
                    #serial move command for diff_step to desired wavelength
                    ser = serial.Serial('COM3', 
                                    baudrate = 9600, 
                                    timeout = None,
                                    xonxoff = True,
                                    parity = serial.PARITY_NONE,
                                    stopbits = serial.STOPBITS_ONE,
                                    bytesize = serial.EIGHTBITS,
                                    )
                    ser.close()
                    ser.open()                            
                    ser.write(b' \r'); 
                    ser.read_until(size=None) 
                    ser.close()
                    print("serial communication initialized")

                    ser.open()
                    ser.write(diff_step); 
                    ser.read_until(size=None)
                    ser.close()
                    print('scan controller is moving by', diff_step, 'steps')
                    print(f"Moving scan controller {diff_wave}nm to {wavelist[0]}nm")
                    #check or wait to reach next wavelength
                    print(f"Holding for current exposure {exposure[idx]} seconds")
                    #take exposure/hold time for exposure
                    print(f"The exposure for {currentwave}nm was taken")
                    
                    ser.open()
                    ser.write(b'@ \r'); #soft stop
                    ser.read_until(size=None)
                    ser.close()
                    print("scan controller stopped")
                    #update
                    if idx < len(wavelist)-1:
                        currentwave = wavelist[idx+1]
        except:
            print("Error with serial communication or movement.")
    except:
        print("Error with user input.")
    finally:
        ser.open()
        ser.write(b'P \r'); #exit program
        ser.read_until(size=None)
        ser.close()
        print("exited program")


In [7]:
try:
    ser = serial.Serial('COM3', 
                            baudrate = 9600, 
                            timeout = None,
                            xonxoff = True,
                            parity = serial.PARITY_NONE,
                            stopbits = serial.STOPBITS_ONE,
                            bytesize = serial.EIGHTBITS,
                            )
    ser.close()
except:
    print("Error, could not connect to controller for auxillary window.")
        
def param(event):
    try:
        ser.open()
        ser.write(b'X \r'); #X=K(ramp speed),I(starting velocity),V(scanning velocity)
        s = ser.read_until(size=None) #reads the data coming from the serial until there is no data left
        param = codecs.decode(s)   
        print("ramp speed, start vel., scan vel. : ", param)
        ser.close()
    except:
        print("Error, could not return parameters.")

def exep(event):
    try:
        ser.open()
        ser.write(b'G \r'); #exectues program stored in non-volitile memory
        s=ser.read_until(size=None)
        print("executing program")
        #add user putting in their program's address
        #address = [input("Enter your stored program's address")]
        ser.close()
    except:
        print("Error, could not execute program")

def wait(event):
    try:
        ser.open
        ser.write(b'W \r'); #wait n milliseconds n= 0 to 65535
        ser.read_until(size=None)
        print("wait n millisecods for next command")
        ser.close()
    except:
        print("Error, could not insert wait command.")

def movestat(event):
    try:
        ser.open()
        ser.write(b'^ \r'); #read moving status 0=no motion, 1=moving, 2=High const. vel., 16=slewing ramping complete
        s=ser.read_until(size=None)
        read = codecs.decode(s)
        print("moving status: ", read)
        ser.close()
    except:
        print("Error, could not read moving status.")

def status(event):
    try:
        ser.open()
        ser.write(b'] \r'); #read limit switch status 0=no limit, 32=home limit, 64=high limit, 128=low limit
        s=ser.read_until(size=None)
        stat = codecs.decode(s)
        stat = int(float(str(stat[4:])))
        print("limit status is: ", stat)
        ser.close()
    except:
        print("Error, could not read limit status.")

def eprg(event):
    try:
        ser.open()
        s=ser.write(b'P \r'); #enter and exit program mode. P0 thru P1000 sets scanner in internal program mode
        s=ser.read_until(size=None)
        print("exited program")
        ser.close()
    except:
        print("Error, could not exit program. Please exit manually.")

def edge(event):
    try:
        ser.open()
        ser.write(b'F4500,0 \r'); #find edge. home swtich must be blocked. motor moves upward 4500steps/sec
        s=ser.read_until(size=None)
        print("finding edge of home flag")
        ser.close()
    except:
        print("Error, could not execute home flag finding function.")

def hcircuit(event):
    try:
        ser.open()
        ser.write(b'A8 \r'); #enable home circuit
        s=ser.read_until(size=None)
        print("home circuit enabled")
        ser.close()
    except:
        print("Error, could not enable home circuit.")

def dcircuit(event):
    try:
        ser.open()
        ser.write(b'A0 \r'); #Disable Home Circuit
        s=ser.read_until(size=None)
        print("disabled home circuit")
        ser.close()
    except:
        print("Error, could not disable home circuit. Check serial connection.")

def acircuit(event):
    try:
        ser.open()
        ser.write(b'A24 \r'); #home accuracy circut enabled
        s=ser.read_until(size=None)
        print("high accuracy circuit enabled")
        ser.close()
    except:
        print("Error, could not enable high accuracy circuit.")

def store(event):
    try:
        ser.open()
        ser.write(b'S \r'); #store parameters to non-volitile memory
        ser.read_until(size=None)
        print("storing parameters to memory")
        ser.close()
    except:
        print("Error, could not store new parameters to memory.")

def init(event):
    try:
        ser.open()                            
        ser.write(b' \r'); 
        ser.read_until(size=None) 
        ser.close()
        print("communication initialized")
    except:
        print("Error, could not initialize communication, check serial connection.")

def clear(event):
    try:
        ser.open()
        ser.write(b'C1 \r'); #clear; erases pre-programmed parameters. only use when unexplainable scanning error has occured
        s=ser.read_until(size=None)
        print("cleared pre-programmed parameters")
        ser.close()
    except:
        print("Error, could not clear parameters.")

def reset(event):
    try:
        ser.open()
        ser.write(b'^C \r'); #Reset; stops motion, sets counter to 0 assumes idle state
        s=ser.read_until(size=None)
        print("reset, stopping motion, becoming idle")
        ser.close()
    except:
        print("Error, could not reset. Could not stop motion.")

def stop(event):
    try:
        ser.open()
        ser.write(b'@ \r'); #soft stop
        s=ser.read_until(size=None)
        print("stopping scan controller")
        ser.close()
    except:
        print("Error, could not stop scan. Check if serial is opened or closed.")

Error, could not connect to controller for auxillary window.


VII. Scan Window function takes previously defined types of scanning movements that require user input and button selection to process. Places them in a window and will have an output for the user to know what their function did or if there wasw an error.

def scanwindow(event):
    try:
        scan = Tk()
        scan.title("All Scans")
        scan.geometry("800x600")
        
        ser = serial.Serial('COM3', 
                                    baudrate = 9600, 
                                    timeout = None,
                                    xonxoff = True,
                                    parity = serial.PARITY_NONE,
                                    stopbits = serial.STOPBITS_ONE,
                                    bytesize = serial.EIGHTBITS,
                                    )
        ser.close()
        #create label for one GoTo movement
        label_Goto = tk.Label(scan,text="Go To Wavelength: ", font="times")
        label_Goto.grid(column=0,row=1) #places label in window

        #creates label for begin and end movement
        label_scans = tk.Label(scan,text="Enter Beginning Wavelength:",font="times")
        label_scans.grid(column=0,row=2)
        label_scane = tk.Label(scan,text="Enter End Wavelength:",font="times")
        label_scane.grid(column=0,row=3)

        #creates label for advanced scan
        label_advscan = tk.Label(scan,text="Enter Wavelengths for Scan: ", font="times")
        label_advscan.grid(column=0,row=4)
        label_expscan = tk.Label(scan,text="Enter Exposure Times for Each Wavelength in Scan", font="times")
        label_expscan.grid(column=0,row=5)

        #creates entry box for entering one wavelength
        Goto_userenter = tk.Entry(scan,width=25)
        Goto_userenter.grid(column=1,row=1) 
        #creates entry box for entering start and end wavelength
        scan_start = tk.Entry(scan,width=25)
        scan_start.grid(column=1,row=2)
        scan_end =tk.Entry(scan,width=25)
        scan_end.grid(column=1,row=3)
        #creates entry box for entering advanced scan wavelength and exposures
        advscan = tk.Entry(scan,width=25)
        advscan.grid(column=1,row=4)
        advexp = tk.Entry(scan,width=25)
        advexp.grid(column=1,row=5)

        #creates button for user to left click to complete GoTo movement
        Gotobutton = tk.Button(scan,text ="Start", bg="yellow")
        Gotobutton.grid(column=2, row=1)#places button in window
        Gotobutton.bind("<Button-1>", GoTo)#connects button to defined GoTo function when clicked
        #creates button for user to left click to complete Scan begin and end movement
        scanbutton = tk.Button(scan,text="Start", bg="blue")
        scanbutton.grid(column=2,row=3)#places button in window
        scanbutton.bind("<Button-1>", scanbasic)#connects button to defined scan function when clicked

        #creates button for user to left click to complete advanced scan movement
        advbutton = tk.Button(scan,text="Start",bg="red")
        advbutton.grid(column=2, row=5) 
        advbutton.bind("<Button-1>", advscan)
    except:
        print("Error, could not open scan window.")
    finally:
        scan.mainloop()

VIII. Auxillary Window function creates many different funcitions that equate to serial commands for monochromator scan settings and movement. 

def auxwin(event):
    try:
        aux = Tk()
        aux.title("Auxillary Functions")
        aux.geometry("1600x900")    
        
        ser = serial.Serial('COM3', 
                                    baudrate = 9600, 
                                    timeout = None,
                                    xonxoff = True,
                                    parity = serial.PARITY_NONE,
                                    stopbits = serial.STOPBITS_ONE,
                                    bytesize = serial.EIGHTBITS,
                                    )
        ser.close()
        
        #Buttons for above auxillary functions.
        parambutton = tk.Button(aux,text ="Parameters",bg="yellow")
        parambutton.grid(column=1,row=1)
        parambutton.bind("<Button-1>", param)
        paramenter = tk.Entry(aux,width=10)
        paramenter.grid(column=2,row=1) 
        exepbutton = tk.Button(aux,text ="Execute", bg="yellow")
        exepbutton.grid(column=3, row=1)#places button in window
        exepbutton.bind("<Button-1>", exep)#connects button to defined GoTo function when clicked
        exepenter = tk.Entry(aux,width=10)
        exepenter.grid(column=4,row=1) 
        waitbutton = tk.Button(aux,text ="Wait Time",bg="yellow")
        waitbutton.grid(column=5,row=1)
        waitbutton.bind("<Button-1>", wait)
        waitenter = tk.Entry(aux,width=10)
        waitenter.grid(column=6,row=1) 
        movestatbutton = tk.Button(aux,text ="Move Status",bg="yellow")
        movestatbutton.grid(column=1,row=2)
        movestatbutton.bind("<Button-1>", movestat)
        statusbutton = tk.Button(aux,text ="Limit Status",bg="yellow")
        statusbutton.grid(column=2,row=2)
        statusbutton.bind("<Button-1>", status)
        eprgbutton = tk.Button(aux,text ="Exit",bg="yellow")
        eprgbutton.grid(column=3,row=2)
        eprgbutton.bind("<Button-1>", eprg)
        edgebutton = tk.Button(aux,text ="Find Edge",bg="yellow")
        edgebutton.grid(column=4,row=2)
        edgebutton.bind("<Button-1>", edge)
        hcircuitbutton = tk.Button(aux,text ="A8",bg="yellow")
        hcircuitbutton.grid(column=1,row=3)
        hcircuitbutton.bind("<Button-1>", hcircuit)
        dcircuitbutton = tk.Button(aux,text ="A0",bg="yellow")
        dcircuitbutton.grid(column=2,row=3)
        dcircuitbutton.bind("<Button-1>", dcircuit)
        acircuitbutton = tk.Button(aux,text ="A24",bg="yellow")
        acircuitbutton.grid(column=3,row=3)
        acircuitbutton.bind("<Button-1>", acircuit)
        storebutton = tk.Button(aux,text ="Store", bg="yellow")
        storebutton.grid(column=4,row=3)
        storebutton.bind("<Button-1>", store)
        initbutton = tk.Button(aux,text ="Initialize",bg="yellow")
        initbutton.grid(column=5,row=3)
        initbutton.bind("<Button-1>", init)
        clearbutton = tk.Button(aux,text ="Clear",bg="yellow")
        clearbutton.grid(column=6,row=3)
        clearbutton.bind("<Button-1>", clear)
        resetbutton = tk.Button(aux,text ="Reset",bg="yellow")
        resetbutton.grid(column=7,row=3)
        resetbutton.bind("<Button-1>", reset)
        stopbutton = tk.Button(aux,text ="Stop",bg="yellow")
        stopbutton.grid(column=8,row=3)
        stopbutton.bind("<Button-1>", stop)
    except:
        print("Error, could not open functions window.")
    finally:
        aux.mainloop()

IX. UI Configuration
Assigns the above function definitions to a button in UI window. Creates labels and user entry boxes and places them in the window. 

#this code creates a separate window for scanning function interactions for monochromator
try:    
    root = Tk() #creates window
    root.title("Scan Controller Operation") #title of window
    root.geometry("1920x1080") #size of window

    #create label for auxillary code functions
    label_aux = tk.Label(root,text="Auxillary Functions", font="times")
    label_aux.grid(column=0,row=1)
    
    #creates button for user to left click to open auxillary functions window
    auxbutton = tk.Button(root,text="",bg="yellow")
    auxbutton.grid(column=0,row=0)
    auxbutton.bind("<Button-1>",auxwin)
    #creates button for user to left click to receive serial connection information
    connectbutton = tk.Button(root,text = "Connect", bg="yellow")
    connectbutton.grid(column=1,row=0) #places button in window
    connectbutton.bind("<Button-1>",connect)#connects button to defined connection function when clicked
    #creates button for user to left click to complete homing movement
    homebutton = tk.Button(root, text = "Home",bg="yellow")
    homebutton.grid(column=2,row=0)#places button in window
    homebutton.bind("<Button-1>",home)#connects button to defined home function when clicked
    #creates button for user to left click to submit preset commands in python code
    #presetbutton = tk.Button(root,text="Enter",bg="yellow")
    #presetbutton.grid(column=1,row=1)#places button in window
    #presetbutton.bind("<Button-1>",AllFunc)
    #creates button for user to left click to open scanning window for taking measurements of wavelength scanning
    scanwindow = tk.Button(root,text="Scan",bg="yellow")
    scanwindow.grid(column=2,row=1)
    scanwindow.bind("<Button-1>",scanwindow)

    #creates entry box for entering one prev. defined command
    preset_userenter = tk.Entry(root,width=25)
    preset_userenter.grid(column=1,row=3)
    #code for output box here
    text_box = tk.Text(root,width = 25,height=2)
    text_box.grid(column=1,row=4)
    text_box.insert("end-1c", "output")
except:
    print("Error, UI Not Functioning, Please Restart.")
finally:
    root.mainloop()#must call before running any tkinter method

In [8]:
#testing output for code
import tkinter as tk
import random

root = tk.Tk()

entry_label = tk.Label(root, text = "Guess a number between 1 and 5: ")
entry_label.grid(row = 0, column = 0)

#Entry field for user guesses.
user_entry = tk.Entry(root)
user_entry.grid(row = 0, column = 1)

text_box = tk.Text(root, width = 25, height = 2)
text_box.grid(row = 1, column = 0, columnspan = 2)

text_box.insert("end-1c", "simple guessing game!")

random_num = random.randint(1, 5)

def guess_number(event = None):
    #Get the string of the user_entry widget
    guess = user_entry.get() 

    if guess == str(random_num):
        text_box.delete(1.0, "end-1c") # Clears the text box of data
        text_box.insert("end-1c", "You win!") # adds text to text box

    else:
        text_box.delete(1.0, "end-1c")
        text_box.insert("end-1c", "Try again!")

        user_entry.delete(0, "end")
# binds the enter widget to the guess_number function
# while the focus/cursor is on the user_entry widget
user_entry.bind("<Return>", guess_number) 

root.mainloop() 



In [9]:

def printSomething():
    # if you want the button to disappear:
    # button.destroy() or button.pack_forget()
    for x in range(9): # 0 is unnecessary
        label = Label(root, text= str(x))
    # this creates x as a new label to the GUI
        label.pack() 

root = Tk()

button = Button(root, text="Print Me", command=printSomething)
button.pack()

root.mainloop()

In [ ]:
from tkinter import *
import numpy as np
import codecs
import serial
from icecream import ic
import tkinter as tk
import time

class Scanwindow:
    def __init__(self):
        scan = Tk()
        scan.title("All Scans")

        #create label for one GoTo movement
        label_Goto = tk.Label(scan,text="Go To Wavelength: ", font="times")
        label_Goto.grid(column=0,row=1) #places label in window

        #creates label for begin and end movement
        label_scans = tk.Label(scan,text="Enter Beginning Wavelength:",font="times")
        label_scans.grid(column=0,row=2)
        label_scane = tk.Label(scan,text="Enter End Wavelength:",font="times")
        label_scane.grid(column=0,row=3)

        #creates label for advanced scan
        label_advscan = tk.Label(scan,text="Enter Wavelengths for Scan: ", font="times")
        label_advscan.grid(column=0,row=4)
        label_expscan = tk.Label(scan,text="Enter Exposure Times for Each Wavelength in Scan", font="times")
        label_expscan.grid(column=0,row=5)

        #creates entry box for entering one wavelength
        Goto_userenter = tk.Entry(scan,width=25)
        Goto_userenter.grid(column=1,row=1) 
        #creates entry box for entering start and end wavelength
        scan_start = tk.Entry(scan,width=25)
        scan_start.grid(column=1,row=2)
        scan_end =tk.Entry(scan,width=25)
        scan_end.grid(column=1,row=3)
        #creates entry box for entering advanced scan wavelength and exposures
        advscan = tk.Entry(scan,width=25)
        advscan.grid(column=1,row=4)
        advexp = tk.Entry(scan,width=25)
        advexp.grid(column=1,row=5)

        #creates button for user to left click to complete GoTo movement
        Gotobutton = tk.Button(scan,text ="Start", bg="yellow")
        Gotobutton.grid(column=2, row=1)#places button in window
        Gotobutton.bind("<Button-1>", GoTo)#connects button to defined GoTo function when clicked
        #creates button for user to left click to complete Scan begin and end movement
        scanbutton = tk.Button(scan,text="Start", bg="blue")
        scanbutton.grid(column=2,row=3)#places button in window
        scanbutton.bind("<Button-1>", scanbasic)#connects button to defined scan function when clicked
        #creates button for user to left click to complete advanced scan movement
        advbutton = tk.Button(scan,text="Start",bg="red")
        advbutton.grid(column=2, row=5) 
        advbutton.bind("<Button-1>", advscan)

        scan.mainloop()

class Auxwindow:
    def __init__(self):
        aux = Tk()
        aux.title("Auxillary Functions")
        aux.geometry("1920x1080")
        
        #Buttons for above auxillary functions.
        label_param = tk.Label(aux,text="parameters",font="times")
        label_param.grid(column=0,row=0)
        aux.mainloop()

class Mainwindow:
    def __init__(self):
        root = Tk()
        root.title("Scan Controller Operations")
        scanbutton = tk.Button(root, text="Scan",bg="yellow")
        scanbutton.grid(column=1,row=1)
        scanbutton.bind("<Button-1>",scanwindow)
        auxbutton = tk.Button(root, text="Aux",bg="yellow")
        auxbutton.grid(column=1,row=2)
        auxbutton.bind("<Button-1>",auxwindow)
        root.mainloop()
        

In [1]:
import codecs
import time
import serial
import numpy as np
from tkinter import *
import tkinter as tk
from icecream import ic

try:
    root = Tk()
    root.title("Scan Controler Operations")
    root.geometry("1920x1080")

    def scanwindow(event):
        try:
            scan = Tk()
            scan.title("All Scans")
            scan.geometry("800x600")

            #create label for one GoTo movement
            label_Goto = tk.Label(scan,text="Go To Wavelength: ", font="times")
            label_Goto.grid(column=0,row=1) #places label in window

            #creates label for begin and end movement
            label_scans = tk.Label(scan,text="Enter Beginning Wavelength:",font="times")
            label_scans.grid(column=0,row=2)
            label_scane = tk.Label(scan,text="Enter End Wavelength:",font="times")
            label_scane.grid(column=0,row=3)

            #creates label for advanced scan
            label_advscan = tk.Label(scan,text="Enter Wavelengths for Scan: ", font="times")
            label_advscan.grid(column=0,row=4)
            label_expscan = tk.Label(scan,text="Enter Exposure Times for Each Wavelength in Scan", font="times")
            label_expscan.grid(column=0,row=5)

            #creates entry box for entering one wavelength
            Goto_userenter = tk.Entry(scan,width=25)
            Goto_userenter.grid(column=1,row=1) 
            #creates entry box for entering start and end wavelength
            scan_start = tk.Entry(scan,width=25)
            scan_start.grid(column=1,row=2)
            scan_end =tk.Entry(scan,width=25)
            scan_end.grid(column=1,row=3)
            #creates entry box for entering advanced scan wavelength and exposures
            advscan = tk.Entry(scan,width=25)
            advscan.grid(column=1,row=4)
            advexp = tk.Entry(scan,width=25)
            advexp.grid(column=1,row=5)

            #creates button for user to left click to complete GoTo movement
            Gotobutton = tk.Button(scan,text ="Start", bg="yellow")
            Gotobutton.grid(column=2, row=1)#places button in window
            Gotobutton.bind("<Button-1>", GoTo)#connects button to defined GoTo function when clicked
            #creates button for user to left click to complete Scan begin and end movement
            scanbutton = tk.Button(scan,text="Start", bg="blue")
            scanbutton.grid(column=2,row=3)#places button in window
            scanbutton.bind("<Button-1>", scanbasic)#connects button to defined scan function when clicked
            #creates button for user to left click to complete advanced scan movement
            advbutton = tk.Button(scan,text="Start",bg="red")
            advbutton.grid(column=2, row=5) 
            advbutton.bind("<Button-1>", advscan)

        except:
            print("Error,could not open scan window")
        finally:
            scan.mainloop()

    def auxwindow(event):
        try:
            aux = Tk()
            aux.title("Auxillary Functions")
            aux.geometry("1920x1080")
            
            #Buttons for above auxillary functions.
            label_param = tk.Label(aux,text="parameters",font="times")
            label_param.grid(column=0,row=0)
        except:
            print("Error, could not open functions window.")
        finally:
            aux.mainloop()

    scanbutton = tk.Button(root, text="Scan",bg="yellow")
    scanbutton.grid(column=1,row=1)
    scanbutton.bind("<Button-1>",scanwindow)
    auxbutton = tk.Button(root, text="Aux",bg="yellow")
    auxbutton.grid(column=1,row=2)
    auxbutton.bind("<Button-1>",auxwindow)
except:
    print("Error, could not open command window.")
finally:
    root.mainloop()

#emergency manual stop 
import serial

ser = serial.Serial('COM4', 
                    baudrate = 9600, 
                    timeout = None,
                    xonxoff = True,
                    parity = serial.PARITY_NONE,
                    stopbits = serial.STOPBITS_ONE,
                    bytesize = serial.EIGHTBITS,
                    )
ser.close()

ser.open()
ser.write(b'@ \r'); #soft stop
s = ser.read_until(size=None)
ser.close()
print("input manually")

def scanadv():
    wavelist= np.array([200, 250, 275, 280, 300, 326, 400],dtype=float)
    exposure= np.array([10, 20, 30, 40, 20, 10, 8, 50],dtype=float)
    """This is a primitive scan function that takes a list of wavelengths and exposure duration list to scan and take expousres. This function assumes that the monochromator is already on the start position"""
    home = 631.26 #nm
    rev = 9000 #steps = 1 nm
    diff_wave = np.roll(wavelist,-1)-wavelist
    diff_step = rev*diff_wave[:-1]
    currentwave = home
    for idx,step in enumerate(diff_step):  
        if idx == 0:
            diff_wave = wavelist[0]-currentwave
            print(f"Moving to {wavelist[0]} now")
            print(f"Hold for current exposure {exposure[idx]}")
            print(f"The exposure for {wavelist[0]} was taken")
            currentwave = wavelist[idx+1]
        else:#command or function to move steps.
            print(f"Moving to {currentwave} now")
            #check or wait to reach next wavelenght
            print(f"Hold for currenet exospure {exposure[idx]}")
            #take exposure/hold time for exposure
            print(f"The exposure for {currentwave} was taken")
            currentwave = wavelist[idx+1]

start_wave = 200.0
end_wave = 400.0 
interval = 11 
exposure_constant = 40 
wavelist = np.linspace(start_wave,end_wave,num=interval)
exposure = np.ones(len(wavelist))*exposure_constant
print(wavelist,exposure)
